In [132]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [133]:
corpus = np.array(['O Coordinator vai enviar as métricas para o Kafka. O Kafka possui 3 réplicas, cada uma delas contendo 1 Cluster manager e um Backend Server . O Cluster manager vai trabalhar com os HDFS enquanto o Backend vai alimentar o Blobstore.',
          'MetricsDB tem 3 componentes principais. O Cluster Manager é responsável por atribuir partições a servidores backend. HDFS é usado para armazenar mapeamentos de partições para servidores. Os servidores de back-end são responsáveis ​​pelo processamento de métricas para um pequeno número de partições. Cada servidor de back-end mantém as últimas duas horas de dados para todas as métricas na memória e verifica os dados na memória a cada duas horas para armazenamento durável, Blobstore. Os coordenadores são responsáveis ​​por rotear solicitações para todos os conjuntos de réplicas e valida que as restrições de quorum desejadas sejam atendidas. MetricsDB é compatível com várias zonas. MetricsDB tem um fator de replicação de três. O produtor Apache Kafka tem uma opção para solicitações em lote, o que nos ajudou a reduzir o número de solicitações para a fila e armazenamento.'])

### 1) Aplicar lowercase e remover caracteres

In [134]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [135]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['coordinator vai enviar as métricas para kafka kafka possui réplicas cada uma delas contendo cluster manager um backend server cluster manager vai trabalhar com os hdfs enquanto backend vai alimentar blobstore'
 'metricsdb tem componentes principais cluster manager responsável por atribuir partições servidores backend hdfs usado para armazenar mapeamentos de partições para servidores os servidores de backend são responsáveis \u200b\u200bpelo processamento de métricas para um pequeno número de partições cada servidor de backend mantém as últimas duas horas de dados para todas as métricas na memória verifica os dados na memória cada duas horas para armazenamento durável blobstore os coordenadores são responsáveis \u200b\u200bpor rotear solicitações para todos os conjuntos de réplicas valida que as restrições de quorum desejadas sejam atendidas metricsdb compatível com várias zonas metricsdb tem um fator de replicação de três produtor apache kafka tem uma opção para solicitações em lote 

### 2) Separar em array de palavras

In [118]:
def text_all(text):
    text_set = set()
    for w in [words.split() for words in text]:
        text_set.update(w)
    return np.array(list(text_set))

vocabulary = text_all(corpus_clear)
#vocabulary

### 3) Remover stopwords

In [136]:
from nltk.corpus import stopwords
filtered_words = [t for t in vocabulary if not t in stopwords.words("portuguese")]

In [137]:
#print(" ".join(filtered_words))
filtered_words

['cada',
 'opção',
 'armazenar',
 'todas',
 'manager',
 'componentes',
 'contendo',
 'cluster',
 'últimas',
 'várias',
 'apache',
 'quorum',
 'ajudou',
 'fator',
 'kafka',
 'dados',
 'coordenadores',
 'restrições',
 'processamento',
 'valida',
 'produtor',
 'durável',
 'coordinator',
 'usado',
 'pequeno',
 'atendidas',
 'hdfs',
 'responsável',
 'zonas',
 'server',
 'réplicas',
 'servidores',
 'responsáveis',
 'reduzir',
 'memória',
 'atribuir',
 'possui',
 'duas',
 'três',
 'enviar',
 'blobstore',
 'backend',
 'enquanto',
 'partições',
 'trabalhar',
 '\u200b\u200bpelo',
 'verifica',
 'solicitações',
 'vai',
 'compatível',
 'todos',
 'mapeamentos',
 '\u200b\u200bpor',
 'conjuntos',
 'métricas',
 'número',
 'lote',
 'rotear',
 'mantém',
 'horas',
 'alimentar',
 'desejadas',
 'replicação',
 'servidor',
 'armazenamento',
 'fila',
 'principais',
 'metricsdb']

In [121]:
def fit_transform(text, words=filtered_words):
    return [int(word in text.split()) for word in words]

features = np.array(list(map(fit_transform, corpus_clear)))
#print(features)

### Usando similaridade do cosseno

In [122]:
def cosine_similarity(v, w):
    return np.dot(v, w)/np.sqrt(np.dot(v, v)*np.dot(w, w))    
    #return np.dot(v, w)/(np.linalg.norm(v)*np.linalg.norm(w)) # usando distância linear

In [123]:
def text_similar(id_text, features=features, text=corpus, n_text=3):
    simillarity = [[cosine_similarity(features[id_text], feature), int(i)] for i, feature in enumerate(features)]
    simillarity = np.array(sorted(simillarity, key=lambda sim: sim[0], reverse=True))    
    return [[text[y], simillarity[x, 0]] for x, y in enumerate(np.int0(simillarity[1:,1]), 1)][:n_text]

In [124]:
print('Texto analisado -> ',corpus[1], '\n')
for t, s in text_similar(1):
    print('Texto: {} | Similaridade: {}'.format(t, round(s, 5)))

Texto analisado ->  MetricsDB tem 3 componentes principais. O Cluster Manager é responsável por atribuir partições a servidores backend. HDFS é usado para armazenar mapeamentos de partições para servidores. Os servidores de back-end são responsáveis ​​pelo processamento de métricas para um pequeno número de partições. Cada servidor de back-end mantém as últimas duas horas de dados para todas as métricas na memória e verifica os dados na memória a cada duas horas para armazenamento durável, Blobstore. Os coordenadores são responsáveis ​​por rotear solicitações para todos os conjuntos de réplicas e valida que as restrições de quorum desejadas sejam atendidas. MetricsDB é compatível com várias zonas. MetricsDB tem um fator de replicação de três. O produtor Apache Kafka tem uma opção para solicitações em lote, o que nos ajudou a reduzir o número de solicitações para a fila e armazenamento. 

Texto: O Coordinator vai enviar as métricas para o Kafka. O Kafka possui 3 réplicas, cada uma delas

### Usando a biblioteca sklearn

In [138]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [126]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = CountVectorizer(analyzer='word')
#features = vectorizer.fit_transform(corpus_clear).todense()
features = vectorizer.fit_transform(corpus_clear)
tokens = vectorizer.get_feature_names()

features.toarray()
create_dataframe(features.toarray(),tokens)

,ajudou,alimentar,apache,armazenamento,armazenar,as,atendidas,atribuir,backend,blobstore,...,três,um,uma,usado,vai,valida,verifica,várias,zonas,últimas
text_1,0,1,0,0,0,1,0,0,2,1,...,0,1,1,0,3,0,0,0,0,0
text_2,1,0,1,2,1,3,1,1,3,1,...,1,2,1,1,0,1,1,1,1,1


In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(corpus_clear, analyzer='word', stop_words='english')
tokens2 = Tfidf_vect.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens2)

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])

#tokens2

,doc_1,doc_2
text_1,1.000000,0.193925
text_2,0.193925,1.000000


In [128]:
def text_similar_sklearn(id_text, features=features, text=corpus, n_text=3):

    simillarity = [[cosine_similarity(features[id_text], feature)[0,0], int(i)] for i, feature in enumerate(features)]
    simillarity = np.array(sorted(simillarity, key=lambda sim: sim[0], reverse=True))    
    return [[text[y], simillarity[x, 0]] for x, y in enumerate(np.int0(simillarity[1:,1]), 1)][:n_text]

In [129]:
print('Texto analisado -> ',corpus[1], '\n')
for t, s in text_similar_sklearn(1):
    print('Texto: {} | Similaridade: {}'.format(t, round(s, 5)))

Texto analisado ->  MetricsDB tem 3 componentes principais. O Cluster Manager é responsável por atribuir partições a servidores backend. HDFS é usado para armazenar mapeamentos de partições para servidores. Os servidores de back-end são responsáveis ​​pelo processamento de métricas para um pequeno número de partições. Cada servidor de back-end mantém as últimas duas horas de dados para todas as métricas na memória e verifica os dados na memória a cada duas horas para armazenamento durável, Blobstore. Os coordenadores são responsáveis ​​por rotear solicitações para todos os conjuntos de réplicas e valida que as restrições de quorum desejadas sejam atendidas. MetricsDB é compatível com várias zonas. MetricsDB tem um fator de replicação de três. O produtor Apache Kafka tem uma opção para solicitações em lote, o que nos ajudou a reduzir o número de solicitações para a fila e armazenamento. 



IndexError: index 2 is out of bounds for axis 0 with size 2